In [498]:
from pathlib import Path
from itertools import cycle


import yfinance as yf

import pandas as pd


import plotly.express as px

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [499]:
import plotly.io as pio

pio.renderers.default = "browser"

In [500]:
# Load S&P 500 data
csv_file = Path("sp500.csv")
if not csv_file.exists():
    df = yf.download("^GSPC")

    df = df.reset_index()  # Use an integer index
    df.columns = df.columns.droplevel(1)  # Remove the second level (ticker)
    df.to_csv(csv_file)
else:
    df = pd.read_csv(csv_file)

df = df.rename(columns={"Date": "date", "Close": "value"})
df = df[["date", "value"]]
df["date"] = pd.to_datetime(df["date"])

In [501]:
# All drawdowns over threshold
threshold = -0.10

df["cummax"] = df["value"].cummax()
df["start"] = df.groupby("cummax")["date"].transform("first")
df["end"] = df.groupby("cummax")["date"].transform("last")

min_indices = df.groupby("cummax")["value"].transform("idxmin")
df["bottom"] = df.loc[min_indices, "date"].values
df["drawdown"] = (df.groupby("cummax")["value"].transform("min") - df["cummax"]) / df["cummax"]

drawdowns = df[df["drawdown"] < threshold].groupby("cummax").max().reset_index()
drawdowns = drawdowns[["start", "bottom", "end", "drawdown"]]

drawdowns["to_bottom"] = drawdowns["bottom"] - drawdowns["start"]
drawdowns["to_recovery"] = drawdowns["end"] - drawdowns["start"]

drawdowns

,start,bottom,end,drawdown,to_bottom,to_recovery
0,1928-05-14,1928-06-12,1928-08-27,-0.102740,29 days,105 days
1,1929-09-16,1932-06-01,1954-09-21,-0.861896,989 days,9136 days
2,1955-09-23,1955-10-11,1955-11-11,-0.105851,18 days,49 days
3,1956-08-02,1957-10-22,1958-09-23,-0.214746,446 days,782 days
4,1959-08-03,1960-10-25,1961-01-26,-0.140175,449 days,542 days
5,1961-12-12,1962-06-26,1963-08-30,-0.279736,196 days,626 days
6,1966-02-09,1966-10-07,1967-05-03,-0.221773,240 days,448 days
7,1967-09-25,1968-03-05,1968-04-26,-0.101137,162 days,214 days
8,1968-11-29,1970-05-26,1972-03-03,-0.360616,543 days,1190 days
9,1973-01-11,1974-10-03,1980-07-16,-0.482036,630 days,2743 days


In [502]:
fig = px.scatter(
    drawdowns,
    title="S&P 500 Drawdowns",
    x="to_bottom",
    y="to_recovery",
    log_x=True,
    log_y=True,
    labels={
        "to_bottom": "Days until Market Bottom",
        "to_recovery": "Days until Market Recovery",
        "drawdown": "Max Drawdown (%)",
    },
)
fig.show()

In [503]:
fig = make_subplots(
    rows=1,
    cols=1,
    specs=[[{"type": "scatter"}]],
    subplot_titles=("S&P 500 Drawdowns",),
    x_title="Days",
    y_title="Drawdown (%)",
)
fig.update_layout(
    yaxis_tickformat=".2%",
    margin_l=100
)
fig.update_annotations(selector=dict(text="Drawdown (%)"), xshift=-70)

for drawdown, color in zip(drawdowns.iloc[:-1].iloc, cycle(px.colors.qualitative.Plotly)):
    mask = (drawdown.start <= df["date"]) & (df["date"] <= drawdown.end)
    period = df[mask].copy()

    period["i"] = (period["date"] - drawdown.start).dt.days
    period["pct"] = (period["value"] - period["value"].iloc[0]) / period["value"].iloc[0]

    fig.add_trace(
        go.Scatter(
            x=period["i"],
            y=period["pct"],
            name=f"{drawdown.start.date()}, {drawdown.to_bottom.days} bottom, {drawdown.to_recovery.days} recovery",
            mode="lines",
            line=dict(width=0.5, color=color),
        )
    )
    # fig.add_trace(
    #     go.Scatter(
    #         x=[drawdown.to_bottom.days, drawdown.to_recovery.days],
    #         y=[drawdown.drawdown, 0],
    #         mode="markers",
    #         name=f"{drawdown.start.date()} ({drawdown.to_bottom.days} day to bottom, {drawdown.to_recovery.days} day to recovery)",
    #         marker=dict(size=4, color=color),
    #         showlegend=False,
    #     )
    # )


drawdown = drawdowns.iloc[-1]
mask = (drawdown.start <= df["date"]) & (df["date"] <= drawdown.end)
period = df[mask].copy()

period["i"] = (period["date"] - drawdown.start).dt.days
period["pct"] = (period["value"] - period["value"].iloc[0]) / period["value"].iloc[0]

fig.add_trace(
    go.Scatter(
        x=period["i"],
        y=period["pct"],
        name=f"{drawdown.start.date()}",
        mode="lines",
        line=dict(width=1, color="black"),
    )
)

fig.show()